In [203]:
import agentpy as ap
import matplotlib.pyplot as plt
import IPython
import random

In [204]:
class Car(ap.Agent):
    
    def setup(self):

        self.idCar = 0
        self.firstCar = 0

def Cars(cars,street,gridSize):
    
    left = int(((gridSize - street) / 2) + 2)
    right = int(left + 1)
    top = int(gridSize - 1)
    pos = []

    for car in cars:
        if car.idCar == 1:
            pos.append((right,0))
        if car.idCar == 2:
            pos.append((left,top))
        if car.idCar == 3:
            pos.append((0,left))
        if car.idCar == 4:
            pos.append((top,right))

    return pos

class Road(ap.Agent):
    
    def setup(self):
        
        self.idRoad = 0

def Roads(size, gridSize):

    right = int(((gridSize - size) / 2) + 2)
    left = int(right + 1)
    pos = []

    for i in range(gridSize):
        pos.append((left, i))

    for i in range(gridSize):
        pos.append((right, i))

    for i in range(gridSize):
        pos.append((i, right))

    for i in range(gridSize):
        pos.append((i, left))

    return pos

In [205]:
class RoadIntersection(ap.Model):
    
    def setup(self):
        n = 0

        self.grass = ap.AgentList(self, int(self.p.gridSize / 2))
        self.roads = ap.AgentList(self, int(self.p.gridSize * 4), Road)
        self.cars = ap.AgentList(self, self.p.numCars ** 2, Car)

        for car in self.cars:
            id = random.randint(1,4)
            car.idCar = id
            car.firstCar = id
        
        numRoad = 1

        for road in self.roads:
            if n < self.p.gridSize:
                road.idRoad = numRoad
                n += 1
            else:
                numRoad += 1
                road.idRoad = numRoad
                n = 1
        
        self.grid = ap.Grid(self, [self.p.gridSize] * 2)
        carPos = Cars(self.cars, self.p.size, self.p.gridSize)
        roadPos = Roads(self.p.size, self.p.gridSize)
        self.grid.add_agents(self.cars, carPos)
        self.grid.add_agents(self.roads, roadPos)

        # CONDICIONES
        self.grass.condition = 0
        self.cars.condition = 1
        self.roads.condition = 2

    def step(self):
        
        right = int(((self.p.gridSize - self.p.size) / 2) + 2)
        left = int(right + 1)

        for car in self.cars:
            neighbors = self.grid.neighbors(car)
            for neighbor in neighbors:
                if neighbor.condition == 2:
                    if car.idCar == 1:
                        if neighbor.idRoad == 1:
                            self.grid.move_by(car,(0,1))
                            [y,x] = self.grid.positions[car]
                            if y == left and x == right:
                                car.idCar = random.choice([1,4])
                            elif y == left and x == left and car.firstCar != 3:
                                car.idCar = random.choice([1,3])
                            break
                    if car.idCar == 2:
                        if neighbor.idRoad == 2:
                            self.grid.move_by(car,(0,-1))
                            [y,x] = self.grid.positions[car]
                            if y == right and x == left:
                                car.idCar = random.choice([2,4])
                            elif y == right and x == right and car.firstCar != 4:
                                car.idCar = random.choice([1,3])
                            break
                    if car.idCar == 3:
                        if neighbor.idRoad == 3:
                            self.grid.move_by(car,(1,0))
                            [y,x] = self.grid.positions[car]
                            if y == right and x == right:
                                car.idCar = random.choice([2,3])
                            elif y == left and x == right and car.firstCar != 2:
                                car.idCar = random.choice([1,3])
                            break
                    if car.idCar == 4:
                        if neighbor.idRoad == 4:
                            self.grid.move_by(car,(-1,0))
                            [y,x] = self.grid.positions[car]
                            if y == left and x == left:
                                car.idCar = random.choice([2,3])
                            elif y == right and x == left and car.firstCar != 1:
                                car.idCar = random.choice([1,3])
                            break

                    if self.grid.positions[neighbor] != self.grid.positions[car]:
                        break

In [206]:
parameters = {
    'numCars': 5,
    'gridSize' : 10,
    'size': 10,
    'steps': 100,
}

In [207]:
def animation_plot(model, ax):
    attr_grid = model.grid.attr_grid('condition')
    color_dict = {0:'#0C920C', 1:'#FF0000', 2:'#9C9C9C', None:'#0C920C'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Simulación intersección de calles\n" f"Steps: {model.t}")

fig, ax = plt.subplots()
model = RoadIntersection(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=15))